In [1]:
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

You should consider upgrading via the '/Users/jiashenc/Workspace/eva/evadb_venv/bin/python -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
nohup eva_server > eva.log 2>&1 &
('05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF FastRCNNObjectDetector already exists, nothing added.\n', '05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF ArrayCount already exists, nothing added.\n', '05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Crop already exists, nothing added.\n')
('05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Open already exists, nothing added.\n', '05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Similarity already exists, nothing added.\n', '05-07-2023 23:37:56 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Yolo already exists, nothing added.\n')
('Traceback (most recent call last):\n', '  File "/Users/jiashenc/Workspace/eva/evadb_venv/bin/eva_server"

You should consider upgrading via the '/Users/jiashenc/Workspace/eva/evadb_venv/bin/python -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Download data
!wget -nc https://www.dropbox.com/scl/fo/fcj6ojmii0gw92zg3jb2s/h\?dl\=1\&rlkey\=j3kj1ox4yn5fhonw06v0pn7r9 -O reddit-images.zip
!unzip -o reddit-images.zip -d reddit-images

File ‘reddit-images.zip’ already there; not retrieving.


Archive:  reddit-images.zip
mapname:  conversion of  failed
 extracting: reddit-images/search.jpg  
 extracting: reddit-images/g348_d7jgzgf.png  
 extracting: reddit-images/g348_d7jhhs3.jpg  
 extracting: reddit-images/g348_d7jphyc.jpg  
 extracting: reddit-images/g348_d7ju7dq.png  
 extracting: reddit-images/g348_d7jghfb.jpg  
 extracting: reddit-images/g1074_d4n1lmn.png  
 extracting: reddit-images/g1074_d4n60oy.jpg  
 extracting: reddit-images/g1190_cln9xzr.png  
 extracting: reddit-images/g1190_clncot0.jpg  
 extracting: reddit-images/g1190_clndsnu.png  
 extracting: reddit-images/g1209_ct6bf1n.png  
 extracting: reddit-images/g1418_cj3om3h.jpg  


 extracting: reddit-images/g1418_cj3qysz.jpg  
 extracting: reddit-images/g1074_d4n6fgs.jpg  
 extracting: reddit-images/g1190_cln97xm.jpg  
 extracting: reddit-images/g1190_clnavnu.jpg  
 extracting: reddit-images/g1190_clnc4uy.png  
 extracting: reddit-images/g1190_clnce4b.jpg  
 extracting: reddit-images/g1209_ct67oqk.jpg  
 extracting: reddit-images/g1418_cj3r4gw.jpg  
 extracting: reddit-images/g1418_cj3z7jw.jpg  
 extracting: reddit-images/g1074_d4mxztt.png  
 extracting: reddit-images/g1074_d4npo8a.jpg  
 extracting: reddit-images/g1190_clnarjl.png  
 extracting: reddit-images/g1209_ct65pvl.jpg  
 extracting: reddit-images/g1209_ct66erw.jpg  
 extracting: reddit-images/g1209_ct6a0g5.png  
 extracting: reddit-images/g1418_cj3o1h6.png  


In [3]:
# Load all images
cursor.execute("LOAD IMAGE 'tutorials/reddit-images/*.jpg' INTO reddit_dataset")
response = cursor.fetch_all()
print(response)
cursor.execute("LOAD IMAGE 'tutorials/reddit-images/*.png' INTO reddit_dataset")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load IMAGE failed: encountered unexpected error Load IMAGE failed due to no valid files found on path tutorials/reddit-images/*.jpg
@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load IMAGE failed: encountered unexpected error Load IMAGE failed due to no valid files found on path tutorials/reddit-images/*.png


In [4]:
# Create UDF
cursor.execute("""CREATE UDF IF NOT EXISTS SiftFeatureExtractor
                    INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                    OUTPUT (features NDARRAY FLOAT32(1, ANYDIM))
                    TYPE  Classification
                    IMPL  'eva/udfs/sift_feature_extractor.py'""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                          0
0  UDF SiftFeatureExtractor already exists, nothing added.
@query_time: 0.006250583999985793


In [5]:
# Keep track of which image gets the most votes
from collections import Counter
vote = Counter()

In [6]:
"""
Image-level similarity search pipeline. This pipeline creates one vector per image.
Next, we should breakdown steps how we build the index and search similar vectors 
using the index.
"""

'\nImage-level similarity search pipeline. This pipeline creates one vector per image.\nNext, we should breakdown steps how we build the index and search similar vectors \nusing the index.\n'

In [7]:
#1. Create index for the entire image
cursor.execute("""CREATE INDEX reddit_sift_image_index 
                    ON reddit_dataset (SiftFeatureExtractor(data)) 
                    USING HNSW""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Index reddit_sift_image_index already exists.


In [8]:
#2. Search similar vectors
cursor.execute("""SELECT name FROM reddit_dataset ORDER BY
                    Similarity(
                      SiftFeatureExtractor(Open('tutorials/reddit-images/search.jpg')),
                      SiftFeatureExtractor(data)
                    )
                    LIMIT 5""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Failed to open file tutorials/reddit-images/search.jpg


In [9]:
#3. Update votes
res_df = response.as_df()
for i in range(len(res_df)):
    vote[res_df["reddit_dataset.name"][i]] += 1
print(vote)

AssertionError: Response is empty

In [ ]:
"""
Object-level similarity search pipeline. This pipeline detects existing object per
image and then create a vector only from the cropped object. Index is then built on
top of vectors for objects. 

Note: The current support for object-level similarity search pipeline is lacking. We
are actively working on it to improve it.
"""

In [ ]:
#1. Create a secondary table that contains object
cursor.execute("""CREATE MATERIALIZED VIEW reddit_object_dataset (name, data, bboxes, labels)
                    AS SELECT name, data, bboxes, labels FROM reddit_dataset
                      JOIN LATERAL UNNEST(Yolo(data)) AS Obj(labels, bboxes, scores)""")
response = cursor.fetch_all()
print(response)

In [ ]:
#2. Create index from objects
cursor.execute("""CREATE INDEX reddit_sift_object_index
                    ON reddit_object_dataset (SiftFeatureExtractor(Crop(data, bboxes)))
                    USING HNSW""")
response = cursor.fetch_all()
print(response)

In [ ]:
#3. Create a cropped images (We are actively working on features to allow
# us to not do this outside SQL)
cursor.execute("LOAD IMAGE 'tutorials/reddit-images/search.jpg' INTO reddit_search_image_dataset")
response = cursor.fetch_all()
print(response)
cursor.execute("SELECT Yolo(data).bboxes FROM reddit_search_image_dataset")
response = cursor.fetch_all()
print(response)


import cv2
import pathlib

res_df = response.as_df()
bboxes = res_df["yolo.bboxes"][0]

img = cv2.imread("reddit-images/search.jpg")
pathlib.Path("reddit-images/search-object/").mkdir(parents=True, exist_ok=True)
for i, bbox in enumerate(bboxes):
    xmin, ymin, xmax, ymax = bbox
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
    cropped_img = img[ymin:ymax, xmin:xmax]
    cv2.imwrite(f"reddit-images/search-object/search-{i}.jpg", cropped_img)

In [ ]:
#4. Search on the secondary table for object-level similarity search
import os

for path in os.listdir("reddit-images/search-object/"):
    path = "tutorials/reddit-images/search-object/" + path
    cursor.execute(f"""SELECT name FROM reddit_object_dataset ORDER BY
                        Similarity(
                          SiftFeatureExtractor(Open('{path}')),
                          SiftFeatureExtractor(data)
                        )
                        LIMIT 1""")
    response = cursor.fetch_all()
    print(response)
    
    res_df = response.as_df()
    for i in range(len(res_df)):
        vote[res_df["reddit_object_dataset.name"][i]] += 1
    print(vote)

In [ ]:
# !pip install matplotlib
import matplotlib.pyplot as plt

# Display top images
vote_list = list(reversed(sorted([(path, count) for path, count in vote.items()], key=lambda x: x[1])))
img_list = [path for path, _ in vote_list]

fig, ax = plt.subplots(nrows=1, ncols=6, figsize=[18,10])
ax[0].imshow(cv2.imread("reddit-images/search.jpg"))
ax[0].set_title("Search")

for i in range(5):
    axi = ax[i + 1]
    img = cv2.imread("../" + img_list[i])
    axi.imshow(img)
    axi.set_title(f"Top-{i + 1}")

plt.show()